In [1]:
import re
import time
import json
import random
import os
import glob
from warnings import filterwarnings

import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from tqdm import tqdm

filterwarnings("ignore")

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
valid_data = pd.DataFrame()
catalog_data = pd.DataFrame()

In [4]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\polyape1\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [5]:
regions = [
    "leningradskaya_oblast",
    # "moskovskaya_oblast"
]
models = [
    "skoda",
    "audi",
    "honda",
    "volvo",
    "bmw",
    "nissan",
    "infiniti",
    "mercedes",
    "toyota",
    "lexus",
    "volkswagen"
]

## Function block

In [6]:
def get_urls_for_model_in_region(model: str, region: str) -> list:
    '''
    Function that takes model and region and parsing urls for vehicle to list
    '''
    result_list = []
    cnt = True
    page_num = 1
    
    while cnt is True and page_num <= 99:
        try:
            driver.get(f'https://auto.ru/{region}/cars/{model}/used/?page={page_num}')
            next_page = driver.find_element(By.CLASS_NAME, 'ListingItemTitle__link')
            page_links = driver.find_elements(By.CLASS_NAME, 'ListingItemTitle__link')

            for item in page_links:
                result_list.append(item.get_attribute('href'))
            print(f'Parsing {page_num} page for {model} in {region}', end='\r')
            page_num += 1
        except NoSuchElementException:
            cnt = False
            print()
            print(f'There are {page_num - 1} pages by {model}')
    
    return result_list

In [7]:
def get_car_info(url: str) -> list:
    '''
    Function to parse each car URL
    '''
    driver.get(url)
    
    year = int(driver.find_element(By.CLASS_NAME, 'CardInfoRow_year').text.split('\n')[1])
    
    car = driver.find_elements(By.CLASS_NAME,'CardBreadcrumbs__itemText')
    brand = car[3].text
    model = car[4].text
    
    car_url = url
    
    bodytype = driver.find_element(By.CLASS_NAME, 'CardInfoRow_bodytype').text.split('\n')[1]
    kmage = int(re.sub('\D', '', driver.find_element(By.CLASS_NAME, 'CardInfoRow_kmAge').text))
    color = driver.find_element(By.CLASS_NAME, 'CardInfoRow_color').text.split('\n')[1]
    
    engine = driver.find_element(By.CLASS_NAME, 'CardInfoRow_engine').text.split('/')
    
    try:
        engineDisplacement = float(re.findall('(\d+.\d+)', engine[0])[0])
        enginePower = int(re.findall('\d+', engine[1])[0])
        fuelType = engine[2]
    except IndexError:
        engineDisplacement = 0
        enginePower = 0
        fuelType = engine
    
    super_gen = json.loads(driver.find_element(By.ID, 'sale-data-attributes').get_attribute('data-bem'))['sale-data-attributes']
    
    vehicleTransmission = driver.find_element(By.CLASS_NAME, 'CardInfoRow_transmission').text.split('\n')[1]
    drive = driver.find_element(By.CLASS_NAME, 'CardInfoRow_drive').text.split('\n')[1]
    wheel = driver.find_element(By.CLASS_NAME, 'CardInfoRow_wheel').text.split('\n')[1]
    state = driver.find_element(By.CLASS_NAME, 'CardInfoRow_state').text.split('\n')[1]
    owner = driver.find_element(By.CLASS_NAME, 'CardInfoRow_ownersCount').text.split('\n')[1]
    pts = driver.find_element(By.CLASS_NAME, 'CardInfoRow_pts').text.split('\n')[1]
    customs = driver.find_element(By.CLASS_NAME, 'CardInfoRow_customs').text.split('\n')[1]
    
    try:
        owningTime = driver.find_element(By.CLASS_NAME, 'CardInfoRow_owningTime').text.split('\n')[1]
    except NoSuchElementException:
        owningTime = None
    
    try:
        description = driver.find_element(By.CLASS_NAME, 'CardDescriptionHTML').text
    except NoSuchElementException:
        description = None
    
    sell_id = int(re.findall('\d+', driver.find_element(By.CLASS_NAME, 'CardHead__id').text)[0])
    price = int(re.sub('\D', '', driver.find_element(By.CLASS_NAME, 'OfferPriceCaption__price').text))
    
    complect_list = []
    
    complect_data = driver.find_elements(By.CLASS_NAME, 'ComplectationGroupsDesktop__itemList')

    for item in complect_data:
        complect_list.extend(item.text.replace('\n', '').split('•')[1:])
    
    try:
        catalog_url = driver.find_element(By.CLASS_NAME, 'CardCatalogLink').get_attribute('href')
    except NoSuchElementException:
        catalog_url = 'No catalog link'
    
    
    data_dict = {
        'brand': brand, 'model': model, 'year': year, 'bodytype': bodytype, 'kmage': kmage,
        'color': color, 'engineDisplacement': engineDisplacement, 'enginePower': enginePower, 
        'fuelType': fuelType, 'super_gen': super_gen, 'vehicleTransmission': vehicleTransmission,
        'drive': drive, 'wheel': wheel, 'state': state, 'owner': owner, 'pts': pts, 'customs': customs, 
        'owningTime': owningTime, 'description': description, 'sell_id': sell_id, 'price': price, 
        'car_url': car_url, 'catalog_url': catalog_url, 'equipment_dict': complect_list
    }
    
    return data_dict

In [8]:
def get_dicts_from_catalog(url: str) -> list:
    '''
    Function to parse catalog URL
    '''
    driver.get(url)

    car_info_full = driver.find_elements(By.CLASS_NAME, 'list-values')
    
    car_info_dict_ru = {}
    
    for item in car_info_full:
        el = item.text.split('\n')
        for i in range(0, len(el) - 1, 2):
            car_info_dict_ru[el[i]] = el[i + 1]
    
    engine_type_dict = {'бензин': 'GASOLINE', 'дизель': 'DIESEL', 
                        'гибрид': 'HYBRID', 'электро': 'ELECTRO', 'газ': 'LPG',
                        'СУГ': 'LPG'}
    gear_type_dict = {'передний': 'FORWARD_CONTROL', 'полный': 'ALL_WHEEL_DRIVE', 'задний': 'REAR_DRIVE'}
    transmission_dict = {'автомат': 'AUTOMATIC', 'робот': 'ROBOT', 
                         'механика': 'MECHANICAL', 'вариатор': 'VARIATOR'}
    
    if car_info_dict_ru['Тип двигателя'] not in ['электро']:
        if car_info_dict_ru.get('Расход топлива, л город/трасса/смешанный') is not None:
            fuel_rate = float(car_info_dict_ru['Расход топлива, л город/трасса/смешанный'].split('/')[2])
        elif car_info_dict_ru.get('Расход топлива, л смешанный') is not None:
            fuel_rate = float(car_info_dict_ru['Расход топлива, л смешанный'])
        elif car_info_dict_ru.get('Расход топлива, л город/смешанный') is not None:
            fuel_rate = float(car_info_dict_ru['Расход топлива, л город/смешанный'].split('/')[1])
        elif car_info_dict_ru.get('Расход топлива, л город/трасса') is not None:
            fuel_rate = sum(map(float, car_info_dict_ru['Расход топлива, л город/трасса'].split('/'))) / 2
        else:
            fuel_rate = None
    else:
        fuel_rate = float(car_info_dict_ru['Запас хода на электричестве, км'])
        
    try:
        clearance_min = min(map(int, re.findall('\d+', car_info_dict_ru['Клиренс'])))
    except KeyError:
        clearance_min = None
    
    car_info_dict_en = {
        'engine_type': engine_type_dict[car_info_dict_ru['Тип двигателя']],
        'gear_type': gear_type_dict[car_info_dict_ru['Привод']],
        'transmission': transmission_dict[car_info_dict_ru['Коробка']],
        'power': int(re.findall('\d+', car_info_dict_ru['Мощность'])[0]),
        'power_kvt': int(re.findall('\d+', car_info_dict_ru['Максимальная мощность, л.с./кВт при об/мин'])[1]),
        'acceleration': car_info_dict_ru.get('Разгон до 100 км/ч, с'),
        'clearance_min': clearance_min,
        'fuel_rate': fuel_rate
    }
    
    url2 = url.replace('specifications', 'equipment')
    
    driver.get(url2)

    equipment_list = []

    equipment_data = driver.find_elements(By.CLASS_NAME, 'catalog__package-list-i')
    for item in equipment_data:
        equipment_list.append(item.text)
    
    car_data = json.loads(driver.find_element(By.CLASS_NAME, 'search-form-v2-mmm').get_attribute('data-bem'))
    equip_data = json.loads(driver.find_element(By.CLASS_NAME, 'catalog__section').get_attribute('data-bem'))
    attrib_data = json.loads(driver.find_element(By.CLASS_NAME, 'sale-data-attributes').get_attribute('data-bem'))
    
    return {'catalog_url': url, 'super_gen_2': car_info_dict_en, 'super_gen_3': attrib_data, 'complectation_dict': equipment_list}

## Gathering URLs from MODEL pages in Regions and getting main info from car page

In [9]:
if not os.path.exists('data/car_data'):
    os.makedirs('data/car_data')
    
for region in regions:
    for model in models:
        model_urls = get_urls_for_model_in_region(model, region)
        for url in tqdm(model_urls):
            time.sleep(random.random())
            try:
                valid_data = valid_data.append(get_car_info(url), ignore_index=True)
            except: # NoSuchElementException:
                print('Error 404', end='\r')
        
        valid_data.to_csv(f'data/car_data/{model}_{region}_car_data.csv')
        valid_data = pd.DataFrame()

Parsing 26 page for skoda in leningradskaya_oblast
There are 26 pages by skoda


  8%|███▊                                             | 74/946 [05:45<47:49,  3.29s/it]

 12%|█████▌                                          | 110/946 [08:35<49:17,  3.54s/it]

 14%|██████▍                                         | 128/946 [10:03<45:41,  3.35s/it]

 20%|█████████▍                                      | 187/946 [14:25<52:23,  4.14s/it]

...

 90%|█████████████████████████████████████    | 1346/1490 [1:50:55<07:00,  2.92s/it]

 94%|██████████████████████████████████████▌  | 1401/1490 [1:54:08<03:42,  2.50s/it]

 98%|████████████████████████████████████████ | 1455/1490 [1:57:24<01:38,  2.82s/it]

 98%|████████████████████████████████████████▎| 1467/1490 [1:58:03<00:59,  2.59s/it]

100%|█████████████████████████████████████████| 1490/1490 [1:59:19<00:00,  4.81s/it]


## Getting additional information from catalog (links from main DF)

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 '2022-03-19_train-test_comparison.ipynb',
 '2022-03-31_train-test_EDA.ipynb',
 '20220327_model_counts.ipynb',
 'data',
 'lib',
 'LICENSE',
 'README.md',
 'requirements.txt',
 'UsedCars_Project_Module_6.ipynb',
 'UsedCars_Project_Module_6_getting_spb_cars.ipynb',
 'UsedCars_Project_Module_6_parser.ipynb',
 'UsedCars_Project_Module_6_parser_20220401.ipynb',
 'venv']

In [27]:
car_csvs = [c for c in os.listdir("data/car_data/") if c.endswith(".csv")]
df = pd.DataFrame()

for c in car_csvs:
    print("Reading", c, end=" ")
    df = df.append(pd.read_csv(f"data/car_data/{c}"))
    print("done")

df.set_index("Unnamed: 0", inplace=True)
df.reset_index(drop=True, inplace=True)
df.sample(5, random_state=42).T

Reading audi_leningradskaya_oblast_car_data.csv done
Reading bmw_leningradskaya_oblast_car_data.csv done
Reading honda_leningradskaya_oblast_car_data.csv done
Reading infiniti_leningradskaya_oblast_car_data.csv done
Reading lexus_leningradskaya_oblast_car_data.csv done
Reading mercedes_leningradskaya_oblast_car_data.csv done
Reading nissan_leningradskaya_oblast_car_data.csv done
Reading skoda_leningradskaya_oblast_car_data.csv done
Reading toyota_leningradskaya_oblast_car_data.csv done
Reading volkswagen_leningradskaya_oblast_car_data.csv done
Reading volvo_leningradskaya_oblast_car_data.csv done


,4047,8427,5484,3025,7757
brand,Mercedes-Benz,Volvo,Skoda,Mercedes-Benz,Volkswagen
model,C-Класс,S60,Kodiaq,V-Класс,Passat
year,1994,2013,2018,2014,2021
bodytype,седан,седан,внедорожник 5 дв.,минивэн,седан
kmage,330000,123000,146425,108712,25000
color,чёрный,чёрный,серебристый,чёрный,чёрный
engineDisplacement,1.8,2.0,1.4,2.1,1.4
enginePower,122,180,150,163,150
fuelType,Бензин,Бензин,Бензин,Дизель,Бензин
super_gen,"{'asciiCat': 'cars', 'category': 'cars', 'engi...","{'asciiCat': 'cars', 'category': 'cars', 'engi...","{'asciiCat': 'cars', 'category': 'cars', 'engi...","{'asciiCat': 'cars', 'category': 'cars', 'engi...","{'asciiCat': 'cars', 'category': 'cars', 'engi..."


In [33]:
df.brand.unique().tolist()

['Audi',
 'BMW',
 'Honda',
 'Infiniti',
 'Lexus',
 'Mercedes-Benz',
 'Nissan',
 'Skoda',
 'Toyota',
 'Volkswagen',
 'Volvo']

In [35]:
if not os.path.exists('data/catalog_car_data'):
    os.makedirs('data/catalog_car_data')
    
for single_brand in df.brand.unique():
    print("Processing", single_brand)
    for car_url in tqdm(df[df['brand'] == single_brand]['catalog_url'].value_counts().index.tolist()):
        if car_url != 'No catalog link':
            try:
                catalog_data = catalog_data.append(get_dicts_from_catalog(car_url), ignore_index=True)
            except: # NoSuchElementException:
                print('Error 404', end='\r')
    catalog_data.to_csv(f'data/catalog_car_data/{single_brand}_catalog_car_data.csv')
    catalog_data = pd.DataFrame()

Processing Audi


100%|█████████████████████████████████████████████| 309/309 [28:22<00:00,  5.51s/it]


Processing BMW


100%|█████████████████████████████████████████████| 476/476 [40:41<00:00,  5.13s/it]


Processing Honda


...

100%|█████████████████████████████████████████████| 347/347 [26:21<00:00,  4.56s/it]


Processing Volkswagen


100%|█████████████████████████████████████████████| 448/448 [36:39<00:00,  4.91s/it]


Processing Volvo


100%|█████████████████████████████████████████████| 189/189 [15:51<00:00,  5.03s/it]


In [50]:
cars_list = glob.glob('data/car_data/*.csv')
cars_df = pd.concat(map(pd.read_csv, cars_list))

catalog_cars_list = glob.glob('data/catalog_car_data/*.csv') 
catalog_df = pd.concat(map(pd.read_csv, catalog_cars_list))

In [51]:
del cars_df['Unnamed: 0']
del catalog_df['Unnamed: 0']
cars_df.reset_index(drop=True, inplace=True)
catalog_df.reset_index(drop=True, inplace=True)
catalog_df.drop_duplicates(subset=['catalog_url'], inplace=True, ignore_index=True)

In [52]:
merged_df = pd.merge(cars_df, catalog_df, on='catalog_url', how='left')

In [53]:
merged_df.sample(3).T

,6254,5724,84
brand,Toyota,Skoda,Audi
model,Corolla,Octavia,A6 allroad
year,2017,2013,2020
bodytype,седан,лифтбек,универсал 5 дв.
kmage,102100,140000,13832
color,чёрный,белый,коричневый
engineDisplacement,1.6,1.4,3.0
enginePower,122,140,249
fuelType,Бензин,Бензин,Дизель
super_gen,"{'asciiCat': 'cars', 'category': 'cars', 'engi...","{'asciiCat': 'cars', 'category': 'cars', 'engi...","{'asciiCat': 'cars', 'category': 'cars', 'engi..."


In [54]:
merged_df.to_pickle('data/20220401_spb_parsed_data.pkl.zip', compression='zip')

In [55]:
merged_df.shape

(8549, 27)